<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/12.%20web%E3%82%A2%E3%83%97%E3%83%AA%E3%82%92%E4%BD%9C%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Streamlit test app**

準備：ngrok (https://ngrok.com/) にサインアップして、auth_tokenを取得

##**サンプルアプリ**

In [ ]:
# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q

import streamlit as st
import os
from pyngrok import ngrok

# シンプルなアプリの作成
with open('test_app.py', 'w') as f:
    f.write('''
import streamlit as st

def main():
    st.title('テストアプリ')
    st.write('これは動作確認用の簡単なアプリです')

    # カウンターの初期化
    if 'count' not in st.session_state:
        st.session_state.count = 0

    # ボタンを作成
    if st.button('カウントアップ'):
        st.session_state.count += 1

    # カウントを表示
    st.write(f'現在のカウント: {st.session_state.count}')

if __name__ == '__main__':
    main()
''')

# Streamlitアプリの起動
def run_streamlit():
    # ポート設定
    port = 8501

    # 既存のプロセスを終了
    os.system('kill -9 $(lsof -t -i:8501) 2>/dev/null')
    print("既存のプロセスを終了しました")

    # ngrokの設定
    ngrok.set_auth_token("2sCdLvAsRQOSaPd9ueedbcUiwun_3r6x2mJeUP1ryELVb3DjE")  # あなたのトークンを設定

    # 既存のトンネルを閉じる
    ngrok.kill()

    # Streamlitの起動
    os.system(f'streamlit run test_app.py --server.port {port} &')

    # ngrokトンネルの作成
    public_url = ngrok.connect(port, "http")
    print(f'\n公開URL: {public_url}')

if __name__ == '__main__':
    run_streamlit()

既存のプロセスを終了しました

公開URL: NgrokTunnel: "https://f52e-34-138-175-39.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q
import os
import random
from pyngrok import ngrok
import subprocess

# Streamlitアプリケーションの作成
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
from PIL import Image
import os

def main():
    st.title('画像表示アプリ')
    st.write('bokujyou.jpgを表示します')

    # 画像パスの設定
    image_path = '/content/drive/MyDrive/AI_laboratory_course/bokujyou.jpg'

    try:
        # 画像の存在確認と表示
        if os.path.exists(image_path):
            image = Image.open(image_path)
            st.image(image, caption='bokujyou.jpg', use_column_width=True)
        else:
            st.error(f'指定された画像が見つかりません。パス: {image_path}')
            st.write('現在のディレクトリ内のファイル:')
            current_dir = os.path.dirname(image_path)
            if os.path.exists(current_dir):
                files = os.listdir(current_dir)
                for file in files:
                    st.write(file)
            else:
                st.error(f'ディレクトリが見つかりません: {current_dir}')

    except Exception as e:
        st.error(f'画像の読み込み中にエラーが発生しました: {str(e)}')

if __name__ == '__main__':
    main()
''')

print("\\n=== Streamlitアプリの起動とngrokの設定を開始 ===")

# Streamlitアプリの起動とngrokの設定
def run_streamlit_with_ngrok():
    # ポート設定
    port = 8501

    # 既存のStreamlitプロセスを終了
    try:
        subprocess.run('kill -9 $(lsof -t -i:8501)', shell=True, check=True)
        print("既存のStreamlitプロセスを終了しました")
    except subprocess.CalledProcessError:
        print("既存のStreamlitプロセスはありません")

    # ngrokの設定
    print("\\n=== ngrokの設定を開始 ===")
    ngrok.set_auth_token('enter ngrok token') # ご自身のトークンに置き換えてください

    # 既存のngrokトンネルを閉じる
    ngrok.kill()
    print("既存のngrokトンネルを終了しました")

    # Streamlitの起動
    print("\\n=== Streamlitアプリを起動します ===")
    subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', str(port)])

    # 新しいngrokトンネルを作成
    try:
        public_url = ngrok.connect(port, "http").public_url
        print(f'\\n=== アプリケーションの準備が完了しました ===')
        print(f'公開URL: {public_url}')
    except Exception as e:
        print(f"ngrokトンネルの作成中にエラーが発生しました: {str(e)}")

if __name__ == '__main__':
    run_streamlit_with_ngrok()

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
\n=== Streamlitアプリの起動とngrokの設定を開始 ===
既存のStreamlitプロセスはありません
\n=== ngrokの設定を開始 ===
既存のngrokトンネルを終了しました
\n=== Streamlitアプリを起動します ===
\n=== アプリケーションの準備が完了しました ===
公開URL: https://33e1-34-56-96-130.ngrok-free.app


In [ ]:
!streamlit run app.py --server.port 8501



2025-02-03 02:19:38.558 Port 8501 is already in use


##**機械学習アプリ**

In [ ]:
# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q
import os
import random
from pyngrok import ngrok

# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q

# Streamlitアプリケーションの作成
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
from PIL import Image
import random

def get_random_prediction():
    # ランダムな確率を生成（合計が1になるように）
    apple_prob = random.random()
    strawberry_prob = 1 - apple_prob

    return {
        "class": "apple" if apple_prob > strawberry_prob else "strawberry",
        "probabilities": {
            "apple": apple_prob,
            "strawberry": strawberry_prob
        }
    }

def main():
    st.title('🍎 リンゴ・イチゴ 判別アプリ 🍓')
    st.write('画像をアップロードして、リンゴかイチゴかを判定します')

    uploaded_file = st.file_uploader("画像をアップロードしてください", type=['png', 'jpg', 'jpeg'])

    if uploaded_file is not None:
        # 画像の表示
        image = Image.open(uploaded_file)
        st.image(image, caption='アップロードされた画像', use_column_width=True)

        # ランダムな判定結果を取得
        result = get_random_prediction()

        # 結果の表示
        st.write('## 判定結果')
        st.write(f'この画像は「**{result["class"]}**」です')

        # 確率の表示
        st.write('### 確率')
        for fruit, prob in result["probabilities"].items():
            st.write(f'{fruit}: {prob*100:.2f}%')
            st.progress(prob)

if __name__ == '__main__':
    main()
''')

print("\n=== Streamlitアプリの起動とngrokの設定を開始 ===")

# Streamlitアプリの起動とngrokの設定
def run_streamlit_with_ngrok():
    # ポート設定
    port = 8501

    # 既存のプロセスを終了
    os.system('kill -9 $(lsof -t -i:8501) 2>/dev/null')
    print("既存のStreamlitプロセスを終了しました")

    # ngrokの設定
    print("\n=== ngrokの設定を開始 ===")
    ngrok.set_auth_token('enter ngrok token')  # ここにあなたのngrokのauthトークンを設定してください

    # 既存のngrokトンネルを閉じる
    ngrok.kill()
    print("既存のngrokトンネルを終了しました")

    # Streamlitの起動
    print("\n=== Streamlitアプリを起動します ===")
    os.system(f'streamlit run app.py --server.port {port} &')

    # 新しいngrokトンネルを作成
    try:
        public_url = ngrok.connect(port, "http")
        print(f'\n=== アプリケーションの準備が完了しました ===')
        print(f'公開URL: {public_url}')
    except Exception as e:
        print(f"ngrokトンネルの作成中にエラーが発生しました: {str(e)}")

if __name__ == '__main__':
    run_streamlit_with_ngrok()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 848.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.7 MB/s eta 0:00:00

=== Streamlitアプリの起動とngrokの設定を開始 ===
既存のStreamlitプロセスを終了しました

=== ngrokの設定を開始 ===
既存のngrokトンネルを終了しました

=== Streamlitアプリを起動します ===

=== アプリケーションの準備が完了しました ===
公開URL: NgrokTunnel: "https://779a-35-231-88-200.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# 必要なパッケージのインストール
!pip install streamlit --q
!pip install pyngrok --q

import os
import sys
from google.colab import drive
import shutil
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from pyngrok import ngrok

# Google Driveのマウントとモデルファイルのコピー
print("=== Google Driveのマウントを開始 ===")
drive.mount('/content/drive')

# モデルファイルのパスを設定
source_path = "/content/drive/MyDrive/AI_laboratory_course/classification.pth"
dest_path = "/content/classification.pth"

# モデルファイルのコピー処理
print("\n=== モデルファイルのコピーを実行 ===")
if os.path.exists(source_path):
    try:
        shutil.copy(source_path, dest_path)
        print(f"モデルファイルのコピーが成功しました")
        print(f"コピー元: {source_path}")
        print(f"コピー先: {dest_path}")
    except Exception as e:
        print(f"モデルファイルのコピー中にエラーが発生しました: {str(e)}")
        sys.exit(1)
else:
    print(f"モデルファイルが見つかりません: {source_path}")
    print("以下のパスを確認してください:")
    !ls -R "/content/drive/MyDrive/AI_laboratory_course"
    sys.exit(1)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 958.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00
=== Google Driveのマウントを開始 ===
Mounted at /content/drive

=== モデルファイルのコピーを実行 ===
モデルファイルのコピーが成功しました
コピー元: /content/drive/MyDrive/AI_laboratory_course/classification.pth
コピー先: /content/classification.pth


In [ ]:
# ※content配下にclassification.pthがコピーされていることを確認してください
# Streamlitアプリケーションの作成
with open('app.py', 'w') as f:
    f.write('''
import os
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# モデルの定義を元の構造に合わせて修正
def load_model():
    model_path = '/content/classification.pth'

    # 直接ResNet18をロード
    model = models.resnet18(pretrained=False)
    # FCレイヤーを置き換え
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    # 重みをロード
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()

    return model

def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    return image.unsqueeze(0)

def main():
    st.title('🍎 リンゴ・イチゴ 判別アプリ 🍓')
    st.write('画像をアップロードして、リンゴかイチゴかを判定します')

    try:
        print("=== モデルのロードを開始 ===")
        model = load_model()
        classes = ["apple", "strawberry"]
        print("=== モデルのロードが完了 ===")

        uploaded_file = st.file_uploader("画像をアップロードしてください", type=['png', 'jpg', 'jpeg'])

        if uploaded_file is not None:
            print("=== 画像の処理を開始 ===")
            image = Image.open(uploaded_file)
            st.image(image, caption='アップロードされた画像', use_column_width=True)

            try:
                input_tensor = preprocess_image(image)
                print("画像の前処理が完了しました")

                with torch.no_grad():
                    output = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(output[0], dim=0)
                    predicted_class = torch.argmax(probabilities).item()
                print("予測が完了しました")

                st.write('## 判定結果')
                st.write(f'この画像は「**{classes[predicted_class]}**」です')

                st.write('### 確率')
                for i, prob in enumerate(probabilities):
                    st.write(f'{classes[i]}: {prob.item()*100:.2f}%')
                    st.progress(prob.item())

            except Exception as e:
                st.error('画像の処理中にエラーが発生しました。')
                st.error(f'エラー詳細: {str(e)}')
                print(f"画像処理エラー: {str(e)}")

    except Exception as e:
        st.error('モデルの読み込み中にエラーが発生しました。')
        st.error(f'エラー詳細: {str(e)}')
        print(f"モデルロードエラー: {str(e)}")

if __name__ == '__main__':
    main()
''')

print("\n=== Streamlitアプリの起動とngrokの設定を開始 ===")

# Streamlitアプリの起動とngrokの設定
def run_streamlit_with_ngrok():
    # ポート設定
    port = 8501

    # 既存のプロセスを終了
    os.system('kill -9 $(lsof -t -i:8501) 2>/dev/null')
    print("既存のStreamlitプロセスを終了しました")

    # ngrokの設定
    print("\n=== ngrokの設定を開始 ===")
    ngrok.set_auth_token('2sCdLvAsRQOSaPd9ueedbcUiwun_3r6x2mJeUP1ryELVb3DjE')  # ここにあなたのngrokのauthトークンを設定してください

    # 既存のngrokトンネルを閉じる
    ngrok.kill()
    print("既存のngrokトンネルを終了しました")

    # Streamlitの起動
    print("\n=== Streamlitアプリを起動します ===")
    os.system(f'streamlit run app.py --server.port {port} &')

    # 新しいngrokトンネルを作成
    try:
        public_url = ngrok.connect(port, "http")
        print(f'\n=== アプリケーションの準備が完了しました ===')
        print(f'公開URL: {public_url}')
    except Exception as e:
        print(f"ngrokトンネルの作成中にエラーが発生しました: {str(e)}")

if __name__ == '__main__':
    run_streamlit_with_ngrok()


=== Streamlitアプリの起動とngrokの設定を開始 ===
既存のStreamlitプロセスを終了しました

=== ngrokの設定を開始 ===
既存のngrokトンネルを終了しました

=== Streamlitアプリを起動します ===

=== アプリケーションの準備が完了しました ===
公開URL: NgrokTunnel: "https://e58e-34-150-138-116.ngrok-free.app" -> "http://localhost:8501"
